<a href="https://colab.research.google.com/github/DanRepositories/TareaIO-GA/blob/main/TareaIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instancia del problema

## Configuración de las variables del problema

In [20]:
import random
import numpy as np

distance_matrix = [[0, 2451,  713, 1018, 1631, 1374, 2408,  213, 2571,  875, 1420, 2145, 1972],
     [2451,    0, 1745, 1524,  831, 1240,  959, 2596,  403, 1589, 1374,  357,  579],
     [713, 1745,    0,  355,  920,  803, 1737,  851, 1858,  262,  940, 1453, 1260],
     [1018, 1524,  355,    0,  700,  862, 1395, 1123, 1584,  466, 1056, 1280,  987],
     [1631,  831,  920,  700,    0,  663, 1021, 1769,  949,  796,  879,  586,  371],
     [1374, 1240,  803,  862,  663,    0, 1681, 1551, 1765,  547,  225,  887,  999],
     [2408,  959, 1737, 1395, 1021, 1681,    0, 2493,  678, 1724, 1891, 1114,  701],
     [213, 2596,  851, 1123, 1769, 1551, 2493,    0, 2699, 1038, 1605, 2300, 2099],
     [2571,  403, 1858, 1584,  949, 1765,  678, 2699,    0, 1744, 1645,  653,  600],
     [875, 1589,  262,  466,  796,  547, 1724, 1038, 1744,    0,  679, 1272, 1162],
     [1420, 1374,  940, 1056,  879,  225, 1891, 1605, 1645,  679,    0, 1017, 1200],
     [2145,  357, 1453, 1280,  586,  887, 1114, 2300,  653, 1272, 1017,    0,  504],
     [1972,  579, 1260,  987,  371,  999,  701, 2099,  600, 1162, 1200,  504,    0]]
n_genes = 13

## Funcion Objetivo

In [4]:
def calc_fitness(individual):
  global distance_matrix
  distance = distance_matrix[individual[0]][individual[-1]]

  for i in range(1, len(individual)):
    distance += distance_matrix[individual[i - 1]][individual[i]]

  return distance

## Funciones auxiliares

In [5]:
def create_individual(n_genes):
  return random.sample(range(n_genes), n_genes)

In [6]:
def create_population(n_genes, cant_individuals):
  return np.array([create_individual(n_genes) for i in range(cant_individuals)])

In [7]:
def calc_fitness_population(population):
  fitness = [calc_fitness(individual) for individual in population]
  return fitness

In [8]:
def get_best(population, fitness):
  best = population[0]
  F_min = fitness[0]
  length = len(population)

  for index in range(1, length):
    
    if fitness[index] < F_min:
      best = population[index]
      F_min = fitness[index]

  return best, F_min

# Algoritmo Genético

## Definición de las operaciones geneticas


### Operador de selección

In [9]:
def get_min_index(fitness, index_participants):
  index_min = index_participants[0]
  min = fitness[index_participants[0]]

  for index in index_participants:
    if fitness[index] < min:
      index_min = index
      min = fitness[index]

  return index_min

In [10]:
def select(population, fitness, k, n_individuals):
  winners = []

  for i in range(n_individuals):
    index_participants = random.sample(range(n_individuals), k)
    
    winner_index = get_min_index(fitness, index_participants)
    winners.append(population[winner_index])

  return winners

### Operador de cruce

In [11]:
def copy_genes(individual_origin, individual_destiny, initial, quantity):
  index = initial
  while quantity > 0:
    individual_destiny[index] = individual_origin[index]

    index = (index + 1) % n_genes
    quantity -= 1

  return individual_destiny

In [12]:
def make_childrens(father_1, father_2):
  # Se los hijos sin genes todavia
  children_1 = [-1] * n_genes
  children_2 = [-1] * n_genes

  # Cantidad de genes que van a ser heredados a los hijos
  cant_genes_inherited = 4

  # Se selecciona un indice inicial al azar, del cual se empezara a copiar los genes
  initial_index = random.randint(0, n_genes - 1)

  # Se copian los genes heredados
  children_1 = copy_genes(father_1, children_1, initial_index, cant_genes_inherited)
  children_2 = copy_genes(father_2, children_2, initial_index, cant_genes_inherited)

  # Se inicializan los indices
  index = (initial_index + cant_genes_inherited) % n_genes
  index_children_1 = index
  index_children_2 = index

  # Mientras no se de la vuelta completa a los padres, se van copiando genes
  while True:

    # Si el gen actual del padre no esta en el hijo, se copia
    if father_1[index] not in children_2:
      children_2[index_children_2] = father_1[index]
      index_children_2 = (index_children_2 + 1) % n_genes

    if father_2[index] not in children_1:
      children_1[index_children_1] = father_2[index]
      index_children_1 = (index_children_1 + 1) % n_genes

    index = (index + 1) % n_genes

    if index == ((initial_index + cant_genes_inherited) % n_genes):
      break

  return children_1, children_2

In [13]:
def crossing(population, n_individuals, p_cross):
  for i in range(1, n_individuals, 2):
    rand = random.random()

    if rand < p_cross:
      population[i - 1], population[i] = make_childrens(population[i - 1], population[i])

  return population

### Operador de mutación

In [14]:
def make_mutation(individual):
  indexes = random.sample(range(n_genes), 2)
  
  individual[indexes[0]], individual[indexes[1]] = individual[indexes[1]], individual[indexes[0]]

  return individual

In [15]:
def mutate(population, n_individuals, p_mutate):
  for i in range(n_individuals):
    rand = random.random()

    if rand < p_mutate:
      population[i] = make_mutation(population[i])

  return population

## Definición del algoritmo


In [16]:
def run_genetic_algorithm(n_individuals, n_generations, p_cross, p_mutate):
  population = create_population(n_genes, n_individuals)
  fitness = calc_fitness_population(population)

  best, min_fitness = get_best(population, fitness)

  for i in range(n_generations):
    population = select(population, fitness, 3, n_individuals)
    
    population = crossing(population, n_individuals, p_cross)
    
    population = mutate(population, n_individuals, p_mutate)

    fitness = calc_fitness_population(population)

    current_best, current_min_fitness = get_best(population, fitness)

    if current_min_fitness < min_fitness:
      best = current_best
      min_fitness = current_min_fitness

  return best, min_fitness

# Benchmark

In [24]:
run_genetic_algorithm(40, 60, 0.7, 0.3)

([11, 1, 8, 6, 12, 4, 3, 2, 7, 0, 9, 5, 10], 7293)